In [ ]:
from tensorflow.keras.datasets import reuters
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

ModuleNotFoundError: No module named 'tensorflow'

## Prepare

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 200 # Example: max sequence length
train_data = pad_sequences(train_data, maxlen=max_len)
test_data = pad_sequences(test_data, maxlen=max_len)

from tensorflow.keras.utils import to_categorical
num_classes = max(train_labels) + 1 # 46 for Reuters
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

## Build

In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Embedding(input_dim=10000, output_dim=128, input_length=max_len),
    layers.GRU(128), # Using GRU for variety
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(num_classes, activation='softmax') # Output layer for multiclass
])

## Compile

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Train

In [ ]:
history = model.fit(train_data, train_labels,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

## Evaluate

In [ ]:
loss, accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")